# Advanced Python - Building Scalable Applications

## Module 2

#### Python ```threading``` module: a deep-dive
 - Python threading module API
 - Creating and managing threads.
 - An overview on threading module.
 - Using the Thread class and the Timer class.
 - Active threads Vs. Daemon threads.
 - Helper functions in the threading module.


#### Parallel processing using ```multiprocessing``` module
 - Multitasking using multiprocessing.Process
 - Process Vs Thread: performance and design implications.
 - Similarities and differences between ```Process``` and ```Thread``` class API
 - Helper functions in the ```multiprocessing``` module.

### Python ```threading``` module API

In [ ]:
from threading import Thread

##### ```Thread``` constructor arguments:
```python
   t = Thread(
         group=None,     # Not implemented. Leave it as None
         target=None,    # The function to be run as a separate thread
         name=None,      # Set the string name of a thread (defaults to Thread-1...)
         args=(),        # Positional arguments to target function
         kwargs=None,    # Keyword arguments to target function
         *,
         daemon=None     # Set to True to create a daemon thread
   )
```
---

In [ ]:
from threading import Thread
from time import sleep

#t = Thread(None, sleep, "monitor-thread", (30,)) # Not recommended, but it works
t = Thread(target=sleep, name="monitor-thread", args=(30,)) # Recommended way to create a thread

print(t)
t.start()
print(t)
t.join()
print(t)

<Thread(monitor-thread, initial)>
<Thread(monitor-thread, started 12901707776)>
<Thread(monitor-thread, stopped 12901707776)>


In [ ]:
t.name = 'new-thread'
t.name

'new-thread'

In [11]:
t.daemon = True

RuntimeError: cannot set daemon status of active thread

In [12]:
t.daemon

False

In [15]:
t = Thread(target=sleep, name="monitor-thread", args=(30,), daemon=True)
print(t.daemon)
t.daemon = False
print(t.daemon)
t.start()


True
False


In [17]:
t.daemon = True

RuntimeError: cannot set daemon status of active thread

In [10]:
t.is_alive()

False

#### Important ```Thread``` attributes / methods:
```python
    from threading import Thread
    from time import sleep

    t = Thread(target=sleep, args=(60,))   
    # Create a new Thread instance for a function sleep(60)
```

```python   
    t.start()  
    # Starts the Thread (initializes the thread internally via OS and 
    # invokes the run() method as new Thread)
```

```python
    t.join()   
    # Waits indefinitely until the target function 
    # spawned off as a separate thread returns.
```

```python
    t.is_alive()  
    # Returns True if Thread is alive, or False if the 
    # thread completed as the target function returned.
```

---

#### Data Attributes of Thread object

```python
    t.name   
    # Returns the name of the Thread as a string. Can be set
    
    t.daemon 
    # Returns True if initialized as a daemon Thread. 
    # Can be set before the Thread has started.

    t.ident       
    # Returns a unique number to identify 
    # the Thread (on Linux, the pthread_t value)

    t.native_id   
    # Returns the OS-level thread-id 
    # (on Linux the tid of the Thread) 
```

---

#### Obsolete attributes / methods:
  
```python
     t.getName()   # Use t.name instead
     t.setName()
     t.isDaemon()  # Use t.daemon instead
     t.setDaemon()
```
---

### Important helper functions in ```threading``` module:

```python
    
    threading.current_thread()    
    # Returns the instance of current Thread
    
    threading.active_count()      
    # Returns the number of threads alive (len(threading.enumerate()))
    
    threading.enumerate()         
    # Returns the list of thread instances which are alive
    
    threading.main_thread()       
    # Returns the instance of the main thread
```
---

In [ ]:
##### Creating a new ```Thread``` class (OO-style threading)

from threading import Thread
class Counter(Thread):
    def __init__(self, count, delay, *args, **kwargs):
        super().__init__(*args, **kwargs)  # Python 3.3+
        # super(Thread, self).__init__(*args, **kwargs) # Python 3.0 to 3.2
        # Thread.__init__(self, *args, **kwargs)
        self.count = count
        self.delay = delay

    def run(self):
        from time import sleep
        for i in range(self.count):
            print(f"{self.name}: counting {i}")
            sleep(self.delay)

if __name__ == '__main__':
    c1 = Counter(count=5, delay=2, name="Count5")
    c2 = Counter(count=10, delay=1, name="Count10")

    c1.start()
    c2.start()

    c1.join()
    c2.join()
    

In [ ]:
##### Creating a new ```Thread``` to run python function

from threading import Thread, current_thread as current

def counter(count, delay):
    from time import sleep
    t = current()  # Returns the thread instance currently running this function
    for i in range(count):
        print(f"{t.name}: counting {i}")
        sleep(delay)

if __name__ == '__main__':
    c1 = Thread(target=counter, name="Count5", args=(5, 2))
    c2 = Thread(target=counter, name="Count10", kwargs={"count": 10, "delay": 1})

    c1.start()
    c2.start()

    c1.join()
    c2.join()
    

##### Active Threads vs Daemon Threads
 - Active Threads keep the python process active. Python process exits when all active threads terminate. By default the main thread of the python process is an active thread, so are threads created and started using ```Thread``` instances when their daemon attribute is not set.

 - Daemon Threads are automatically terminated when there are no more Active Threads alive in a python process. A daemon thread can be created by setting the ```daemon=True``` argument while constructing the Thread class, or setting the ```.daemon``` attribute to True to a Thread instance before the thread's ```.start()``` method is invoked.

##### Common use-cases for Daemon Threads
 - Heart-beat monitoring threads
 - Housekeeping / routine cleanup threads
 - Most daemon threads are designed to run an infinite loop that waits for an event or timed sleep, wakes up and perform a routine activities

##### Important notes
 1. Caution must be exercised if a daemon thread should be use resources managed by active threads. For example, reading from files which could be closed by active threads can lead to runtime errors in daemon threads that could not be gracefully handled.
 2. It is pointless to ```join()``` on a daemon thread as that will keep the active thread alive and waiting until the daemon thread exits.

---

In [20]:
from threading import Event, current_thread as current

ev = Event()

def testfn():
    t = current()
    print(f"{t.name}: started")
    
    print(f"{t.name}: waiting for Event to be set...")
    ev.wait()
    
    print(f"{t.name}: Event is set, resuming execution...")

t1 = Thread(target=testfn)
t2 = Thread(target=testfn)
t1.start()
t2.start()


Thread-121 (testfn): startedThread-122 (testfn): started
Thread-122 (testfn): waiting for Event to be set...

Thread-121 (testfn): waiting for Event to be set...


In [23]:
from threading import Event, current_thread as current

ev = Event()

def testfn():
    t = current()
    print(f"{t.name}: started")
    
    print(f"{t.name}: waiting for Event to be set...")
    if ev.wait(timeout=10):
        print(f"{t.name}: Event is set, resuming execution...")
    else:
        print(f"{t.name}: Timeout occurred, Event was not set")

t1 = Thread(target=testfn)
t2 = Thread(target=testfn)
t1.start()
t2.start()


Thread-127 (testfn): startedThread-128 (testfn): started
Thread-128 (testfn): waiting for Event to be set...

Thread-127 (testfn): waiting for Event to be set...


In [24]:
ev.set()

Thread-127 (testfn): Event is set, resuming execution...Thread-128 (testfn): Event is set, resuming execution...



In [25]:
from threading import Timer
Timer?

Init signature: Timer(interval, function, args=None, kwargs=None)
Docstring:     
Call a function after a specified number of seconds:

t = Timer(30.0, f, args=None, kwargs=None)
t.start()
t.cancel()     # stop the timer's action if it's still waiting
Init docstring:
This constructor should always be called with keyword arguments. Arguments are:

*group* should be None; reserved for future extension when a ThreadGroup
class is implemented.

*target* is the callable object to be invoked by the run()
method. Defaults to None, meaning nothing is called.

*name* is the thread name. By default, a unique name is constructed of
the form "Thread-N" where N is a small decimal number.

*args* is a list or tuple of arguments for the target invocation. Defaults to ().

*kwargs* is a dictionary of keyword arguments for the target
invocation. Defaults to {}.

*context* is the contextvars.Context value to use for the thread.
The default value is None, which means to check
sys.flags.thread_inherit_con

In Python, threads cannot run in parallel on multi-CPU platforms due to the implementation of its Global-Interpreter-Lock (GIL)

To implement true parallel processing in Python using standard runtime and library -> use multiprocessing instead of threading


---

The ```multiprocessing.Process``` is a drop-in replacement 
for ```threading.Thread``` for most of the API features

However, Process class also provides 
a ```.terminate()``` and a ```.kill()``` method

In [26]:
from multiprocessing import Process
Process?

Init signature:
Process(
    group=None,
    target=None,
    name=None,
    args=(),
    kwargs={},
    *,
    daemon=None,
)
Docstring:     
Process objects represent activity that is run in a separate process

The class is analogous to `threading.Thread`
File:           /opt/anaconda3/envs/myenv/lib/python3.14/multiprocessing/context.py
Type:           type
Subclasses:     

In [27]:
from time import sleep
from multiprocessing import Process

p = Process(target=sleep, name="sleeper", args=(150,))
p

<Process name='sleeper' parent=97161 initial>

#### Standard Process object methods:
  
```python
    p.start()
    p.join()
    p.is_alive()
```
### Standard attributes:

```python

   p.name
   p.daemon
```


In [29]:
p.start()

In [30]:
p.ident

33619

In [31]:
p.pid

33619

In [32]:
p.join(timeout=5)
p.is_alive()

True

In [33]:
p.kill()

In [34]:
p.is_alive()

False

#### Helper functions in ```multiprocessing``` module

```python
    import multiprocessing as mp

    mp.current_process()   
    # Returns the instance of currently running Process
    
    mp.active_children()   
    # Returns a list of active child Processes (equivalent to threading.enumerate)

    mp.cpu_count()
    # Returns the number of available CPUs (same as os.cpu_count())
```

#### Deciding the number of worker processes to create for running compute intensive tasks:

num_workers = mp.cpu_count() // average_duty_cycle_of_tasks


#### Choosing the right strategy for implementing multitasking
  1. Generators
     - Use this for stream processing pipelines (reading and filtering out data from I/O stream in producer-consumer pattern).
     - Example: file readers, Cursors as results in SQL database, etc... 

  2. Coroutines (or Greenlets / Eventlets)
     - For managing asynchronous I/O events in a client-server architecture.
     - Scalable for handling 1000+ asynchronous client requests due to almost zero OS context-switch overhead
     - Do NOT use them for compute-intensive workloads as they are not preemptible within the running Python process.
     - Suitable for tasks with average duty-cycles below 0.005 (or 
       below `sys.getswitchinterval()`)
  
  3. `threading.Thread`
     - For managing few preemptible tasks that could become CPU intensive at some points.
     - Building GUIs.
     - Suitable for tasks with average duty-cycles between 0.005 to 0.2
  
  4. `multiprocessing.Process`
     - For massive computational work-loads that needs to be parallelized in multiple CPUs
     - Suitable for task with average duty-cycles between 0.2 to 1.0



#### Exercise / Homework:
 - Extract all prime numbers from a huge list of integers. Implement the same using
   parallel-processing techniques and compare the overall performance